In [8]:
!pip install pandas
!pip install --upgrade jinja2

In [1]:
%%html
<style> .container { width: 100% !important; }</style>

# common

In [2]:
import pandas as pd
pd.set_option('max_colwidth', 200)  # Set a very large value
pd.set_option('display.max_rows', 200) # Set a very large value

def add_borders(val):
    return 'border: 1px solid black; font-family: Consolas; font-size: 12px;'

def get_dataframe_name(df):
    for name, obj in globals().items():
        if obj is df and isinstance(obj, pd.DataFrame):
            return name

style_df = lambda df : df.style.map(add_borders)
getcols  = lambda df : display(style_df(pd.DataFrame (list(df.columns), columns=[get_dataframe_name(df)]).T))

def get_dataframe_columns():
    # Initialize an empty list to store the data
    data = []

    # Iterate through all variables in the global namespace
    for name, obj in globals().items():
        if isinstance(obj, pd.DataFrame):
            # Get the dataframe's column names
            columns = obj.columns.tolist()
            # Create a list of tuples containing df_name and col_name
            df_col_pairs = [(name, col) for col in columns]
            data.extend(df_col_pairs)

    # Create a new DataFrame from the collected data
    df = pd.DataFrame(data, columns=['df_name', 'col_name'])
    df['val'] = df['col_name']
    df=df[df.df_name.apply(lambda x : str(x).split('_')[-1])=="df" ]
    return df.pivot_table ( index = 'df_name', columns="col_name" , values = "val" , aggfunc = lambda x : ','.join(list (x)) ).fillna("")


--------
# Load Data

## locations_df

In [163]:
locations_df = pd.read_json("./data/LocationsParsed.json")
locations_prd_df = locations_df[locations_df['host'].isin(['odp-fin-prod-hvr-metadata.odp.health.ge.com','odp-us-prod-hvr-metadata-db.cjyhx859wwhg.us-east-1.rds.amazonaws.com'])]
getcols(locations_df)

,0,1,2,3,4,5,6,7,8,9,10,11
locations_df,database,db_instance,db_node_name,dbdesc,host,loc_class,loc_db_user,loc_description,loc_name,loc_remote_login,loc_remote_node,loc_remote_port


In [4]:
locations_df.head(1).T

,0
database,ushvr12
db_instance,usinnovationredshift
db_node_name,us-innovation-redshift.c8ziwm1qxh67.us-east-1.redshift.amazonaws.com
dbdesc,dev
host,odp-us-innovation-postgres-db.cxzrm3enxxmy.us-east-1.rds.amazonaws.com
loc_class,redshift
loc_db_user,504005930
loc_description,tredp
loc_name,tredp
loc_remote_login,isc_dev


## Channelslocations_df

In [5]:
Channelslocations_df = pd.read_json("./data/hvr_Channel2Location.json")
Channelslocations_prd_df = Channelslocations_df[Channelslocations_df['host'].isin(['odp-fin-prod-hvr-metadata.odp.health.ge.com','odp-us-prod-hvr-metadata-db.cjyhx859wwhg.us-east-1.rds.amazonaws.com'])]
getcols(Channelslocations_df)

,0,1,2,3,4,5,6,7
Channelslocations_df,chn_name,act_name,act_parameters,loc_class,loc_name,host,database,dbdesc


In [6]:
Channelslocations_df.head(2).T

,0,1
chn_name,edhr_01,edhr_01
act_name,Capture,Integrate
act_parameters,/NoTruncate,"/RenameExpression=""edhr_test/public/{hvr_tbl_name}/year={hvr_integ_tstamp %Y}/month={hvr_integ_tstamp %m}/day={hvr_integ_tstamp %d}/{hvr_integ_tstamp}.parquet"" /MaxFileSize=999999999"
loc_class,oracle,file
loc_name,sedhp,ts3ui
host,odp-us-innovation-postgres-db.cxzrm3enxxmy.us-east-1.rds.amazonaws.com,odp-us-innovation-postgres-db.cxzrm3enxxmy.us-east-1.rds.amazonaws.com
database,ushvr12,ushvr12
dbdesc,dev,dev


## Channelslocations_pivoted_df

In [160]:
#-- ignore all channels with d_ preffix
#-- count channels per action and pivot
Channelslocations_pivoted_df     = Channelslocations_df[Channelslocations_df.chn_name.apply(lambda x: x.split('_')[0]!='d')].pivot_table(index=['database','host','loc_name'] , columns='act_name',values='chn_name' , aggfunc=len).fillna(0).reset_index()
Channelslocations_pivoted_prd_df =  Channelslocations_pivoted_df[Channelslocations_pivoted_df['host'].isin(['odp-fin-prod-hvr-metadata.odp.health.ge.com','odp-us-prod-hvr-metadata-db.cjyhx859wwhg.us-east-1.rds.amazonaws.com'])]
getcols(Channelslocations_pivoted_df)

,0,1,2,3,4
Channelslocations_pivoted_df,database,host,loc_name,Capture,Integrate


# Sources summary

In [9]:
df = get_dataframe_columns().T.reset_index()

In [86]:
from itertools import product
from itertools import chain

pproduct = lambda x,y : product(y,[x])
denom_dict = lambda d : dict (chain.from_iterable(map( lambda k :list(pproduct(*(k))) ,  list(d.items())) ))

desc={ 'meta'    :['host','database','dbdesc'],
       'location':['loc_class','loc_remote_port','loc_description'],
       'agent'   :['loc_remote_login','loc_remote_node'],
       'action'  :['Capture','Integrate','loc_name','act_name','act_parameters'],
       'db'      :['loc_db_user','db_instance','db_node_name'],
       'channel' :['chn_name']
     }

desc_ = denom_dict(desc)

In [87]:
df['desc'] = df.col_name.apply ( lambda x : desc_.get(x,'_'))
style_df(df.set_index('col_name').sort_values('desc'))

df_name,Channelslocations_df,Channelslocations_pivoted_df,Channelslocations_pivoted_prd_df,Channelslocations_prd_df,locations_df,desc
col_name,,,,,,
Capture,,Capture,,,,action
Integrate,,Integrate,,,,action
act_name,act_name,,act_name,act_name,,action
act_parameters,act_parameters,,act_parameters,act_parameters,,action
loc_name,loc_name,loc_name,loc_name,loc_name,loc_name,action
loc_remote_login,,,,,loc_remote_login,agent
loc_remote_node,,,,,loc_remote_node,agent
chn_name,chn_name,,chn_name,chn_name,,channel
db_instance,,,,,db_instance,db


---------------
# Analysis 
--------------

## channel names homonyms
chanel with same name on different hubs will cause risk of overwirtting if we try to colocate them in same hub

In [100]:
df = Channelslocations_prd_df.groupby(['host','database','chn_name'])[['loc_name']].count().reset_index()
style_df(
df.pivot_table(index=['host','chn_name'],columns=['database'] , values='loc_name' , aggfunc=len,margins=True).fillna('').astype(str).apply(lambda x: x.str.rstrip('.0')).sort_values(['All','host'] , ascending=False).head(84)
    )

## Number of channels to be migrated

In [151]:
len(set (map (lambda x :','.join(x) , Channelslocations_prd_df[['host','database','chn_name']].values.tolist() )))   

1634

--## old defition locations_prod_df = locations_df[locations_df['dbdesc'].isin(['US hvr prod','fin-prod'])]

-- how many locations are using our agents

In [152]:
len(list (map (   lambda x : ",".join(x), Channelslocations_pivoted_df[['host','database','loc_name']].values.tolist())))

1023

In [148]:
len(set (map (   lambda x : ",".join(x), locations_prd_df[['database','host','loc_name']].values.tolist())))

1878

In [19]:
locations_prod_df.groupby(['database','host','loc_name'])['db_instance'].count().reset_index().sort_values('db_instance')

,database,host,loc_name,db_instance
1848,ushvr11,odp-us-prod-hvr-metadata-db.cjyhx859wwhg.us-east-1.rds.amazonaws.com,tgrd1,1
1849,ushvr11,odp-us-prod-hvr-metadata-db.cjyhx859wwhg.us-east-1.rds.amazonaws.com,tgrd2,1
1850,ushvr11,odp-us-prod-hvr-metadata-db.cjyhx859wwhg.us-east-1.rds.amazonaws.com,tgrds,1
1851,ushvr11,odp-us-prod-hvr-metadata-db.cjyhx859wwhg.us-east-1.rds.amazonaws.com,tmski,1
1852,ushvr11,odp-us-prod-hvr-metadata-db.cjyhx859wwhg.us-east-1.rds.amazonaws.com,trace,1
...,...,...,...,...
1874,ushvr11,odp-us-prod-hvr-metadata-db.cjyhx859wwhg.us-east-1.rds.amazonaws.com,vcp,1
1875,ushvr11,odp-us-prod-hvr-metadata-db.cjyhx859wwhg.us-east-1.rds.amazonaws.com,web,1
1876,ushvr11,odp-us-prod-hvr-metadata-db.cjyhx859wwhg.us-east-1.rds.amazonaws.com,web1,1
1877,ushvr11,odp-us-prod-hvr-metadata-db.cjyhx859wwhg.us-east-1.rds.amazonaws.com,zabix,1


In [20]:
len(Channelslocations_pivoted_df) , len(locations_prod_df) # --- are locations repated ? why numbers are different ???

(1023, 1878)

In [22]:
locations_prod_channels_df=pd.merge(locations_prod_df,Channelslocations_pivoted_df  , on=['database','host','loc_name'], how='inner')
len(locations_prod_channels_df)

548

In [23]:
print ('Number of locations per hvr meta database')

locations_prod_channels_df.groupby(['dbdesc','host','database'])['loc_name'].agg(len).reset_index().set_index(['dbdesc','database'])

Number of locations per hvr meta database


host  \
dbdesc      database                                                                         
US hvr prod ushvr00   odp-us-prod-hvr-metadata-db.cjyhx859wwhg.us-east-1.rds.amazonaws.com   
            ushvr02   odp-us-prod-hvr-metadata-db.cjyhx859wwhg.us-east-1.rds.amazonaws.com   
            ushvr03   odp-us-prod-hvr-metadata-db.cjyhx859wwhg.us-east-1.rds.amazonaws.com   
            ushvr04   odp-us-prod-hvr-metadata-db.cjyhx859wwhg.us-east-1.rds.amazonaws.com   
            ushvr05   odp-us-prod-hvr-metadata-db.cjyhx859wwhg.us-east-1.rds.amazonaws.com   
            ushvr07   odp-us-prod-hvr-metadata-db.cjyhx859wwhg.us-east-1.rds.amazonaws.com   
            ushvr08   odp-us-prod-hvr-metadata-db.cjyhx859wwhg.us-east-1.rds.amazonaws.com   
            ushvr09   odp-us-prod-hvr-metadata-db.cjyhx859wwhg.us-east-1.rds.amazonaws.com   
            ushvr10   odp-us-prod-hvr-metadata-db.cjyhx859wwhg.us-east-1.rds.amazonaws.com   
            ushvr11   odp-us-prod-hvr-metadata-db.cjyhx859wwhg.us-east-1.rds.amazonaws.com   
fin-prod    ushvr00                            odp-fin-prod-hvr-metadata.odp.health.ge.com   
            ushvr02                            odp-fin-prod-hvr-metadata.odp.health.ge.com   
            ushvr03                            odp-fin-prod-hvr-metadata.odp.health.ge.com   

                      loc_name  
dbdesc      database            
US hvr prod ushvr00         31  
            ushvr02         13  
            ushvr03         70  
            ushvr04         52  
            ushvr05         26  
            ushvr07         57  
            ushvr08         32  
            ushvr09        191  
            ushvr10         20  
            ushvr11         10  
fin-prod    ushvr00         30  
            ushvr02          6  
            ushvr03         10

print ('Number of locations per hvr meta database')
locations_prod_df.groupby(['dbdesc','host','database'])['loc_name'].agg(len).reset_index().set_index(['dbdesc','database'])

---------  if channel is using single agent for Capture and Single ganent for Integrate ( is is not matter if it is same agent or not ) total number of counts should be same !!

In [26]:
print  ('Integrate remote Nodes')
df2= locations_prod_channels_df.pivot_table(index=['loc_remote_node','loc_class'], columns=['loc_remote_port'], values='Integrate', aggfunc='sum' , margins=True).fillna('_').astype(str).apply(lambda x: x.str.rstrip('.0'))
style_df(

df2 
)


Integrate remote Nodes


In [27]:
df2= locations_prod_channels_df.pivot_table(index=['loc_remote_node','loc_class'], columns=['loc_remote_port'], values='Capture', aggfunc='sum',margins=True).fillna('_').astype(str).apply(lambda x: x.str.rstrip('.0'))
style_df(

df2 
)

In [28]:
print ('number of agents per node (represented as port) - count of locations')

agg_function= lambda x : '<br>'.join(  set(x)) 
df1= locations_prod_df.groupby('loc_remote_node').agg({'database':agg_function})

df2= locations_prod_df.pivot_table(index=['loc_remote_node'], columns=['loc_remote_port'], values='loc_name', aggfunc='count').fillna('').astype(str).apply(lambda x: x.str.rstrip('.0'))
style_df(
#pd.merge(df1, df2, on='loc_remote_node', how='left')
df2   
    )

number of agents per node (represented as port) - count of locations


loc_remote_port,0,4343,4344,5454,6080,8080,8088,8443,9090,9995,18081
loc_remote_node,,,,,,,,,,,
,278,,,,,,,,,,
10.119.217.1,,1,,,,,,,,,
10.229.1.113,,1,7,,,,,,,,
10.229.2.236,,,,,,4,,,,,
10.242.109.196,,539,,,9,7,,13,,1,
10.242.109.235,,,,,,4,,,,,
10.242.109.242,,,,,,,,5,3,5,1
10.242.109196,,4,,,,,,,,,
10.242.112.130,,46,,1,,,,,,,


In [ ]:
-- rac://cgehedq2-scan.am.health.ge.com:1525 repeated multiple times is it same agent ?

In [29]:
style_df (
            locations_prod_df.pivot_table(index='loc_remote_node', columns=['loc_class'], values='loc_name', aggfunc='count').fillna('').astype(str).apply(lambda x: x.str.rstrip('.0'))
         )

loc_class,file,file_locdir,file_s3s,file_sftp,greenplum,kafka,mysql,oracle,postgresql,redshift,salesforce,sqlserver,teradata
loc_remote_node,,,,,,,,,,,,,
,,11,102,7,,4,2,111,31,,4,,6
10.119.217.1,,,,,,,,,,,,1,
10.229.1.113,,2,4,,,,,,,2,,,
10.229.2.236,,,2,,,,,,,2,,,
10.242.109.196,5,368,8,18,2,5,,49,58,35,,,21
10.242.109.235,,,,,,,,,,4,,,
10.242.109.242,,,15,,,,,,21,5,,,
10.242.109196,,4,,,,,,,,,,,
10.242.112.130,,,,,,,,,1,,3,43,


In [30]:
style_df (
locations_prod_df.pivot_table(index='loc_remote_node', columns=['dbdesc','database'], values='loc_name', aggfunc='count').fillna('').astype(str).apply(lambda x: x.str.rstrip('.0'))
)

In [31]:
pd.DataFrame (locations_df.columns, columns=["columns"])

,columns
0,database
1,db_instance
2,db_node_name
3,dbdesc
4,host
5,loc_class
6,loc_db_user
7,loc_description
8,loc_name
9,loc_remote_login


for loc_lass in set(locations_prod_df['loc_class']):
    df = locations_prod_df[locations_prod_df['loc_class']==loc_lass]
    display(df)

## Source Synonyms (ok for target , not ok for source- exception is file)

In [33]:
#//TODO filter singletons

concat_func = lambda x: '</br> '.join(set(map(str, x)))

agg_funcs = {'loc_name': concat_func }


for loc_lass in set(locations_prod_df['loc_class']):

    df = locations_prod_df[locations_prod_df['loc_class']==loc_lass]

    dff = style_df (
              df.pivot_table(index=['loc_class','db_node_name','db_instance','loc_db_user'], columns=['dbdesc'], values=['loc_name'], aggfunc=concat_func).fillna('').astype(str).apply(lambda x: x.str.rstrip('.0')).sort_values(by=['db_node_name'])
            )
    display (dff)

,,,,loc_name
,,,dbdesc,US hvr prod
loc_class,db_node_name,db_instance,loc_db_user,
file,###,###,###,az_lb ofa suath suate az_hf


In [ ]:
-- homonyms -> any locations that exists on different databases but has different meaning
-- should we do loc clean-up before or during migration (or after)
-- are loc_names unique?

## homonymns on db_node_name  & db_instancelevels of locations

In [36]:
concat_func = lambda x :  [len (set(x)) , set(x)]

# Define a filtering function


dff2 = locations_prod_df.groupby(['dbdesc','loc_class','loc_name'])['database'].agg(lambda x : '<br>'.join(set(x)))

def filter_func(x):
    return x['Value'].sum() > 1

locations_prod_df.loc[:, 'db_concat'] = locations_prod_df['db_node_name'] + ' ---> ' + locations_prod_df['db_instance']


df= locations_prod_df.pivot_table(index=['dbdesc','loc_class','loc_name'], values=['db_concat'], aggfunc= concat_func ).fillna('').reset_index()#.astype(str).apply(lambda x: x.str.rstrip('.0'))
dff = df [df.db_concat.apply(lambda x : x[0] >1)].reset_index()
dff.loc[:, 'db_node_name_hmns'] = dff.apply(lambda x:  "</br>".join(x['db_concat'][1]) , axis=1)


dfff = pd.merge(dff, dff2, on=['dbdesc','loc_class','loc_name'], how='inner')


dfff.to_json('location_homonyms.json', orient='records', lines=True)

style_df(
    dfff[['dbdesc','loc_class','loc_name','db_node_name_hmns','database']].set_index(['dbdesc','loc_class','loc_name'])
)

## homonymns on db_node_name levels of locations

concat_func = lambda x :  [len (set(x)) , set(x)]

# Define a filtering function
def filter_func(x):
    return x['Value'].sum() > 1


df= locations_prod_df.pivot_table(index=['dbdesc','loc_class','loc_name'], values=['db_node_name'], aggfunc= concat_func ).fillna('').reset_index()#.astype(str).apply(lambda x: x.str.rstrip('.0'))
dff = df [df.db_node_name.apply(lambda x : x[0] >1)].reset_index()
dff.loc[:, 'db_node_name_hmns'] = dff.apply(lambda x:  "</br>".join(x['db_node_name'][1]) , axis=1)

style_df(
    dff[['dbdesc','loc_class','loc_name','db_node_name_hmns',]].set_index(['dbdesc','loc_class','loc_name'])
)